In [38]:
from dotenv import load_dotenv
load_dotenv()
import logging
from bs4 import BeautifulSoup
import html2text
import httpx
import yaml
import json
from pydantic import Field, BaseModel
from langgraph.graph import MessagesState, StateGraph, START, END
from typing import List
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langgraph.types import Send
import operator
from typing import Annotated
from typing import NamedTuple
from experiments.visualization.dict_to_reactflow import dict_to_tree_positions
from experiments.utils.fetch_docs import fetch_documents
from experiments.phase1_json_dfs import compiler_graph
# from experiments.code_reflection_agent import final_agent


# Set up the logger
logging.basicConfig(
    level=logging.INFO,  # Set to DEBUG for detailed logs
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        # logging.FileHandler("scraper.log"),  # Log to a file
        logging.StreamHandler()  # Log to console
    ]
)

logger = logging.getLogger(__name__)

agent_architecture_urls = ["https://langchain-ai.github.io/langgraph/tutorials/multi_agent/multi-agent-collaboration",
 "https://langchain-ai.github.io/langgraph/tutorials/multi_agent/agent_supervisor",
 "https://langchain-ai.github.io/langgraph/tutorials/multi_agent/hierarchical_agent_teams",
 "https://langchain-ai.github.io/langgraph/tutorials/plan-and-execute/plan-and-execute",
 "https://langchain-ai.github.io/langgraph/tutorials/self-discover/self-discover",
]


template = """Your job is to get information from a user about what kind of agent they wish to build.

You should get the following information from them:

- What the objective of the agent is
- Various usecases of the agent 
- Some examples of what the agent will be doing (Input and expected output pairs)

If you are not able to discern this info, ask them to clarify! Do not attempt to wildly guess.

After you are able to discern all the information, call the tool AgentInstruction"""


class AgentInstructions(BaseModel):
    """Instructions on how to build the Agent"""
    objective: str = Field(description= "What is the primary objective of the agent")
    usecases: List[str] = Field(description= "What are the various responsibilities of the agent which it needs to fulfill")
    examples : str = Field(description= "What are some examples of the usage of the agent (input query and expected output from the agent) ?")

class ArchEvaluationReport(BaseModel):
    """Class to represent the architecture evaluation report"""
    name: str = Field(description="Name of the agent architecture being evaluated")
    highlights: str = Field(description="Concise summary of the architecture in 5 lines")
    evaluation_score: int = Field(description="evaluation of suitability of the agentic_architecture against user requirements from 1-10, 1 being least relevant to 10 being most relevant")
    justification: str = Field( description="Justification for the score")
    tailored_design: str = Field(description="Tailored design using the architecture")
    
class ArchEvaluationWithUrl(NamedTuple):
    url: str
    report: ArchEvaluationReport

class AgentBuilderState(MessagesState):
    agent_instructions: AgentInstructions = Field("the requirement analysis generated by the model.")
    arch_evaluation_reports: Annotated[List[ArchEvaluationWithUrl], operator.add] = Field("list of agent architectures suggested in map-reduce step")
    best_agent_architecture: ArchEvaluationWithUrl = Field("The agent architecture best suited to above requirements")
    json_code: str = Field("The json code generated")
    python_code: str = Field("The Python code generated")
    reactflow_json: str = Field("The ReactFlow code generated")


class ArchitectureEvaluationState(MessagesState):
    agent_instructions: AgentInstructions = Field("the requirement analysis generated by the model.")
    url: str = Field("url of the agent architecture to evaluate against")

llm = ChatOpenAI(temperature=0, model="gpt-4o-mini", streaming=True)

def requirement_analysis_node(state: AgentBuilderState):
    
    llm_with_tool = llm.bind_tools([AgentInstructions])
    response = llm_with_tool.invoke([SystemMessage(content=template)] + state["messages"])
    
      # Construct the final answer from the arguments of the last tool call  
    if len(response.tool_calls) == 0:
        return {"messages": [response]}
    
    agent_instructions = response.tool_calls[0]
    agent_instructions = AgentInstructions(**agent_instructions["args"])
    
    return {"messages": [response], "agent_instructions": agent_instructions}

ARCH_EVALUATION_PROMPT = PromptTemplate.from_template(
    """
You are tasked with assessing the provided agentic architecture documentation to determine its relevance and applicability to the user requirements outlined below.

Inputs:

- Agentic Architecture Documentation:
{agent_architecture}

- User Requirements:- Objectives: {objective}
- Use Cases: {responsibilities}
- Examples: {examples}


Deliverables:
Your evaluation should be presented in the following structured format:

- Architecture Name:
Provide the name of the agent architecture being assessed.
- Concise Identifier:
Suggest a brief, descriptive name for the architecture that encapsulates its purpose.
- Key Highlights (2-3 Lines):
Identify and summarize the most significant aspects or features of the architecture.
- Feature Summary:
Offer a detailed overview of the architecture's unique elements, functionalities, and capabilities.
- Relevance Score (1-10):
Assign a score based on the alignment between the architecture's features and the user's requirements (1 = minimally relevant, 10 = highly relevant).
- Score Justification:
Provide a clear and concise rationale (5-10 sentences) for the relevance score, highlighting how specific features match—or fail to match—the user's objectives, use cases, and examples.
- Implementation Proposal:
Outline a tailored approach for leveraging the architecture to meet the user’s requirements. Be specific and actionable, addressing how it can fulfill the stated objectives and responsibilities.
"""
)



def architecture_evaluation_map_node(state: AgentBuilderState):
    return [Send("evaluate_against_architecture", {"agent_instructions": state["agent_instructions"], "url": url}) for url in agent_architecture_urls]

def route_state(state: AgentBuilderState):
    messages = state["messages"]
    if isinstance(messages[-1], AIMessage) and messages[-1].tool_calls:
        return "add_tool_message"
    elif not isinstance(messages[-1], HumanMessage):
        return END
    return "requirement_analysis"

def evaluate_against_architecture(state: ArchitectureEvaluationState):
    agent_instructions: AgentInstructions = state["agent_instructions"]
    url: str = state["url"]
    llm_with_structured_output = llm.with_structured_output(ArchEvaluationReport)
    archEvaluationReport: ArchEvaluationReport = llm_with_structured_output.invoke(
        [SystemMessage(content=ARCH_EVALUATION_PROMPT.format(agent_architecture=fetch_documents(url),
                                                             objective=agent_instructions.objective,
                                                             responsibilities=agent_instructions.usecases,
                                                             examples = agent_instructions.examples))])
    
    return {
        "messages": [AIMessage(content=f"Evaluated architecture {url}, arch_name: {archEvaluationReport.name}")],
        "arch_evaluation_reports": [ArchEvaluationWithUrl(url,archEvaluationReport)],
    }

def best_architecture(state: AgentBuilderState):
    """Select the best architecture based on the evaluation reports."""
    # Sort the architectures based on their evaluation scores
    arch_reports : List[ArchEvaluationWithUrl] = state["arch_evaluation_reports"]
    sorted_architectures = sorted(arch_reports, key=lambda x: x.report.evaluation_score , reverse=True)
    
    # Select the best architecture (the first one in the sorted list)
    best_architecture = sorted_architectures[0]
    
    print("found the best architecture")
    
    # Return the best architecture as the output
    return {
        "messages": [AIMessage(content="Best architecture selected!")],
        "best_agent_architecture": best_architecture,
    }


AGENT_KERNEL_PROMPT = PromptTemplate.from_template(
    """
        Task Overview:
        Design a langgraph StateGraph object implementing the {agent_architecture_name} architecture, tailored to fulfill the user requirements defined below

        <Requirements>
        Objectives: {objective}
        usecases: {responsibilities}
        examples: {examples}
        </Requirements>

        Provided Documentation:
        Refer to the documentation for {agent_architecture_name} below to guide your implementation:
        <Documentation for {agent_architecture_name}>
        {agent_architecture}
        </Documentation for {agent_architecture_name}>
        
        Implementation Suggestions:
        Apply the {agent_architecture_name} architecture concepts and align them with the user requirements provided. Suggestions on tailoring the implementation:
        {agent_tailored}
        
        Expected Output:
        Your task is to produce a compiled StateGraph object.

        Guidelines for Code Generation:
        - Accuracy: Avoid hallucinations or speculative assumptions when writing code. Refer exclusively to the provided documentation.
        - Understanding: Thoroughly comprehend the architecture and examples of code.
        - Customization: Generate code tailored specifically to meet the user requirements.

    """)

def agent_kernel_builder(state: AgentBuilderState):
    """Build the agent kernel using the best architecture."""
    best_architecture: ArchEvaluationWithUrl = state["best_agent_architecture"]
    agent_instructions : AgentInstructions = state["agent_instructions"]
    langgraph_glossary_url = "https://langchain-ai.github.io/langgraph/concepts/low_level/"
    agent_architecture_url : str = best_architecture.url
    agent_architecture_report : ArchEvaluationReport = best_architecture.report
    # agent_architecture_report.name
    #agent_architecture_report.highlights
    #agent_architecture_report.justification
    #agent_architecture_report.tailored_design
    
    response =  llm.invoke([SystemMessage(content=AGENT_KERNEL_PROMPT.format(
        objective=agent_instructions.objective,
        responsibilities=agent_instructions.usecases,
        examples = agent_instructions.examples,
        # langgraph_glossary=fetch_documents(langgraph_glossary_url),
        agent_tailored=agent_architecture_report.tailored_design,
        agent_architecture_name = agent_architecture_report.name,
        agent_architecture=fetch_documents(agent_architecture_url)))])
    
    # Return the generated agent kernel as the output
    return {
        "messages": [AIMessage(content="Generated agent kernel code!")],
        "python_code": response.content,
    }

CODE_TO_JSON_PROMPT = PromptTemplate.from_template("""
You are tasked with converting the following stategraph comPilation code into a JSON. 

Contextual documents for understanding code:
{documents}

The Input code is as follows:
{code_snippet}

OUTPUT: Explaination and JSON. Do not include any code blocks. Seperate the JSON and explaination blocks and ensure that there is an explaination for each line of JSON produced but keep the blocks seperated.
Each Output JSON will have a nodes sections containing all the nodes and an edges section

Please follow:
1. Produce the explaination first and then the JSON after it. DO not produce the JSON first. 
2. For any conditional edges, please include all the nodes that the source of a conditional edge can reach as part of the explaination.
3. Any Edge entry in the JSON can only be conditional(mention conditional: true) if the source for that edge acts as a source for multiple edges. If you cannot point to atleast 2 targets for 1 source, then that source will not have any conditional edges
4. A source can have any number of targets. Please write the explaination for each source node to target node edge
5. Please ensure that the JSON starts with __START__ node and __END__ node with the correct edges from and to them
6. Ensure all elements in the nodes sections of the output json contain the following fields: Schema_info, input_schema, output_schema, description, function_name. Please do not return any entries in the nodes without these fields and these fields can't be empty
7. Ensure all elements in the edges sections of the output json contain the following fields: source, target, routing_conditions, conditional. Please do not return any entries in the edges without these fields and they can't be empty
8. Every node should be a part of atleast one edge, Please ensure this is followed
9. Attach the code snippet for each node aswell that. Please extract it from the Input code


Example output JSON for a node:
                                             
   {{
        "id": "code_node"
        "schema_info": /"/"/"CodeWriterState:
        type: TypedDict
        fields:
        - name: user_query
          type: str
        - name: execution_result
            type: str/"/"/",
        "input_schema": "CodeWriterState",
        "output_schema":"RequiremenCodeWriterStatetAnalysisState",
        "description":"This node analyzes the user_query, if the query is to write a code, it will make a tool call to run the proposed code. This node returns command object",
        "function_name": "code_step"
    }}

Example output JSON for an edge:
edge:{{ source: "abc", target: "cde", routing_condition: "if abc made a tool call then go to cde", "conditional": true}}
edge:{{ source: "abc", target: "xyz", routing_condition: "if abc made an interupt to a human then go to xyz", "conditional": true}}
edge:{{ source: "xyz", target: "_END_", routing_condition: "no nodes to go after xyz, we have our final output for this path", "conditional": false}}


High level JSON format of the graph

{{
    "nodes": [
        {{ ... }},
        {{ ... }}
    ],
    "edges": [
        {{ ... }},
        {{ ... }}
    ]
}}
""")

def code_to_json_node(state: AgentBuilderState):
    """Convert the generated code to JSON."""
    langgraph_glossary_url = "https://langchain-ai.github.io/langgraph/concepts/low_level/"
    json_code_ouptut = llm.invoke([SystemMessage(content=CODE_TO_JSON_PROMPT.format(
        code_snippet=state["python_code"],
        documents = fetch_documents(langgraph_glossary_url),
        ))])
    
    # Return the JSON code as the output
    return {
        "messages": [AIMessage(content="Generated JSON code!")],
        "json_code": json_code_ouptut.content,
    }


JSON_CODE_COMBINE_PROMPT = PromptTemplate.from_template("""
You are tasked with verifying and updating the provided JSON that represents the nodes and edges of the langgraph to ensure it is correct with respect to the input code.

Contextual Documents for Understanding Code:
{documents}

Input Code:
<Input code>{code_snippet}</Input code>
JSON:
<JSON>{node_json}</JSON>

OUTPUT: JSON
Your task is divided into two parts:
- Validation: Verify that the JSON adheres to the rules outlined below.
- Correction and Augmentation: If the JSON is incorrect or incomplete, update it with a clear justification for every change, and include the code snippet for each node extracted from the input code.

Rules for Validation and Update:
- Conditional Edges:- An edge can be marked as conditional: true only if its source acts as a source for multiple edges (i.e., at least two targets).
- If the source does not meet this condition, then it cannot have conditional edges.

- Edge Targets:- Each source can have any number of targets. This flexibility must be maintained.

- Start and End Nodes:- The JSON must begin with the __START__ node and conclude with the __END__ node, with correct edges to and from them. Edges into the END node can also be conditional if they meet the above mentioned conditions      

- Node Structure:- Each node entry in the nodes section of the JSON must include the following non-empty fields:- schema_info
- id
- schema_info
- input_schema
- output_schema
- description
- function_name


- Edge Structure:- Each edge entry in the edges section of the JSON must include the following non-empty fields:- source
- target
- routing_conditions
- conditional


- Node-Edge Relationship:- Every node must be part of at least one edge. Ensure this relationship is consistently followed.

- Node Code Snippets:- Attach a code field to every node in the JSON, extracted directly from the input code.

- Schema Requirements:- The final JSON must conform to the schema provided below:


Schema for JSON. Ensure the following schema is followed. No field should be missing for any node or edge:
- Node Example:

{{
  "code_node": {{
    "id" : "<id of the node, similar to name>"
    "schema_info": "<define the class structure of the state of a node>",
    "input_schema": "<input state object>",
    "output_schema": "<output state object>",
    "description": "<description>",
    "function_name": "<function_name>",
    "code": "<python_code>"
  }}
}}


- Edge Example:

{{
  "edge": {{
    "source": "<source_node>",
    "target": "<target_node>",
    "routing_condition": "<routing_condition>",
    "conditional": true/false
  }}
}}


Key Instructions:
- Do not update any pre-existing field of the JSON unless you have an extremely strong justification for doing so.
- Clearly document the reasoning behind any additions, updates, or modifications to the JSON. Justifications should draw inspiration from the contextual documents mentioned earlier.
- Ensure conditional edges strictly adhere to the rules outlined above.
- Input_Schema and output_schemas can only have value None in JSON for START and END nodes. Please follow this without fail
- Include a code field for each code_node entry, with the exact code that corresponds to the node in the input code.
""")

json_formatter_doc = """
You will be given a json by the user, along with explanations and markdown, your target is to filter and only output the json schema.

Output: Only serialized JSON, no explanations or markdown, no ``` notations.

the response would be starting with a '{' and ending with '}'"""


def json_better_node(state: AgentBuilderState):
    """Add code to the json flow"""
    langgraph_glossary_url = "https://langchain-ai.github.io/langgraph/concepts/low_level/"
    json_code_ouptut = llm.invoke([SystemMessage(content=JSON_CODE_COMBINE_PROMPT.format(
        code_snippet=state["python_code"],
        documents = fetch_documents(langgraph_glossary_url),
        node_json = state["json_code"]
        ))])

    formatted_json = llm.invoke([SystemMessage(content=json_formatter_doc)]+ [HumanMessage(content=json_code_ouptut.content)])
    reactflowjson = dict_to_tree_positions(json.loads(formatted_json.content))
    # Return the JSON code as the output
    return {
        "messages": [AIMessage(content="Generated updated JSON code!")],
        "json_code": formatted_json.content,
        "reactflow_json" : reactflowjson

    }

In [39]:
from bs4 import BeautifulSoup
import html2text
import httpx

def fetch_documents(url: str) -> str:
    """Fetch a document from a URL and return the markdownified text.

    Args:
        url (str): The URL of the document to fetch.

    Returns:
        str: The markdownified text of the document.
    """
    httpx_client = httpx.Client(follow_redirects=True, timeout=10)

    try:
        response = httpx_client.get(url, timeout=10)
        response.raise_for_status()
        html_content = response
        soup = BeautifulSoup(html_content, 'html.parser')
    
        img_tags = soup.find_all('img')
        for img_tag in img_tags:
            img_tag.decompose()

        target_div = soup.find('div', class_= "theme-doc-markdown markdown") #langchain
        
        if not target_div:
            target_div = soup.find('article') #langraph

        if not target_div:
            target_div = soup.find('html') #langraph

        if not target_div:
            return html2text.html2text(str(soup))
        
        return html2text.html2text(str(target_div))
    except (httpx.HTTPStatusError, httpx.RequestError) as e:
        return f"Encountered an HTTP error: {str(e)}"



In [ ]:
import json
dict_tool_link = json.load( open( "D:\AgentAgent\AgentAgent\experiments\\tool_creation\\tools_link_json.json") )
dict_tool_doc = json.load( open( "D:\AgentAgent\AgentAgent\experiments\\tool_creation\\tools_doc_json.json") )
def lowercase_keys(input_dict):
    """
    Returns a new dictionary with all keys converted to lowercase.
    """
    return {k.lower(): v for k, v in input_dict.items()}

dict_tool_link = lowercase_keys(dict_tool_link)
dict_tool_doc = lowercase_keys(dict_tool_doc)

Initial_prompt = """You are an expert python developer. You will be given a description of a python function. 

You job is to estimate and extract the following information:

- What exactly does this python do. What is the detailed objective of the function. Please write 1-5 lines
- Suggest or extract the name of the the function
- What would be the inputs/arguements required into this function to make it work. Please all mentioned the type of each input
- WHat would be output produced by this input. Please mention the output type 

Here is the description of the function you need to create:
<description>
{desc}
</description>
"""



class FunctionInstructions(BaseModel):
    """Instructions for defining a python function"""
    objective: str = Field(description= "what does this pythion function do")
    name: str = Field(description="name of the python function")
    input : List[str] = Field(description= "what would be the input arguements to this function along with the types")
    output: List[str] = Field(description="what would be the output/return attributes for the function along with the types")
    name_toolkit: str = Field(description="what would be the toolkit/ code SDK that will be used")
    code: str = Field(description="the final python code")
# Annotated[str, operator.add]

class CodebuilderState(BaseModel):
    """Instructions for defining a python function"""
    code: str = Field(description= "tailored code for the python function")


def functional_analysis_node(state: FunctionInstructions):
  print("functional_analysis_node")
  llm_with_structured_output = llm.with_structured_output(FunctionInstructions)
  functionalReport: FunctionInstructions = llm_with_structured_output.invoke(
      [SystemMessage(content=Initial_prompt.format(desc = state.objective))])
  return {  "messages": [AIMessage(content="Generated JSON code!")],
           "objective": functionalReport.objective,
           "name": functionalReport.name,
           "input": functionalReport.input,
           "output": functionalReport.output}

write_code_prompt = """
You are a skilled code generation assistant. Your task is to create executable code using the following information:
- SDK Documentation: The provided documentation outlines the functionalities and usage details of the SDK. Use this as the reference for constructing your code.
- Objective: A clear description of what the code is intended to achieve.
- Input: The expected input for the code (e.g., variables, parameters, data types).
- Output: The desired result or outcome of the code (e.g., format, type, or structure).
- SDK Name: The name of the SDK that must be used in the code.

Your goal is to generate executable code that:
- Adheres to the requirements outlined above.
- Follows standard coding practices and is optimized for readability and efficiency.
- Utilizes the specified SDK appropriately based on the documentation provided.
- Only return a self contained function
- Your output should only contain a code block containing the required function and nothing else. Please do no include any explainantions
- Write your code in python
- Please also provide which API keys will be required and define the API keys as part of the function
- Please also write the doc string for the python function
- Ensure that the function you produce is decorated with @tool. That means its defination should be preceeded by '@tool' in the line above

Here are some details about the python function you will be creating:
<objective>
{objective}
</objective>

<input schema>
{inputs}
</input schema>

<output schema>
{output}
</output schema>

<name of function>
{name}
</name of function>

Documentation for SDK that might be helpful:
<documentation>
{docs}
</documentation>

"""

Best_sdk_prompt = """
You are a highly specialized language model designed to assist in selecting the most suitable SDK for a given use case. You are provided with the following:
- A dictionary containing pairs of SDK names and their respective descriptions.
- Requirements for a piece of code, including the objective, input, and output.

Your task is to:
- Identify the SDK from the provided dictionary whose description best matches the given use case described in the code requirements.
- Also give preferences to SDKs that are generally more well known or are used more frequently in the industry (Use google tools for anything search related)
- Return only the name of the matching SDK without any additional text or formatting.
- Please ensure that the string you return is a valid key of the dictionary you get as input. PLEASE VERIFY THAT THE STRING YOU RETURN EXISTS AS A KEY IN THE INPUT DICTIONARY

Input Example:
Dictionary:
{{
"SDK_A": "[SDK_CC_ABC]Provides tools for web scraping and data extraction.",
"SDK_B": "Enables natural language processing for unstructured text.",
"SDK_C": "Facilitates the integration of payment gateways in applications."
}}
Code Requirements:
Objective: Extract data from multiple web pages.
Input: URLs of the web pages.
Output: Structured data in JSON format.

Expected Output:
SDK_A


Input :
<dictionary>
{dictionary}
</dictionary>

<objective>
{objective}
</objective>

<input schema>
{inputs}
</input schema>

<output schema>
{output}
</output schema>

<name of function>
{name}
</name of function>


"""


def sdk_production_node(state: FunctionInstructions):
    objective_agent: str = state.objective
    name: str = state.name
    input_args : List[str] = state.input
    output_args: List[str] = state.output
    response = llm.invoke([SystemMessage(content=Best_sdk_prompt.format(
          objective=objective_agent,
          inputs=input_args,
          output=output_args,
          name=name,
          dictionary = dict_tool_doc
    ))])
    code_snips = response
    return {
            "name_toolkit": response.content.lower()}

def code_production_node(state: FunctionInstructions):
    objective_agent: str = state.objective
    name: str = state.name
    input_args : List[str] = state.input
    output_args: List[str] = state.output
    toolkit: str = state.name_toolkit
    docs = fetch_documents(dict_tool_link[toolkit])
    response = llm.invoke([SystemMessage(content=write_code_prompt.format(
          objective=objective_agent,
          inputs=input_args,
          output=output_args,
          name=name,
          docs = docs,
    ))])
    code_snips = response
    return {
            "code": response.content}

<>:2: SyntaxWarning: invalid escape sequence '\A'
<>:3: SyntaxWarning: invalid escape sequence '\A'
<>:2: SyntaxWarning: invalid escape sequence '\A'
<>:3: SyntaxWarning: invalid escape sequence '\A'
C:\Users\anupa\AppData\Local\Temp\ipykernel_15480\1586735054.py:2: SyntaxWarning: invalid escape sequence '\A'
  dict_tool_link = json.load( open( "D:\AgentAgent\AgentAgent\experiments\\tool_creation\\tools_link_json.json") )
C:\Users\anupa\AppData\Local\Temp\ipykernel_15480\1586735054.py:3: SyntaxWarning: invalid escape sequence '\A'
  dict_tool_doc = json.load( open( "D:\AgentAgent\AgentAgent\experiments\\tool_creation\\tools_doc_json.json") )


In [67]:
from langgraph.checkpoint.memory import InMemorySaver
workflow = StateGraph(FunctionInstructions)
workflow.add_node("func_analysis", functional_analysis_node)
workflow.add_node("sdk_write", sdk_production_node)
workflow.add_node("code_write", code_production_node)
checkpointer = InMemorySaver()
workflow.add_edge("code_write", END)
workflow.add_edge("sdk_write","code_write")
workflow.add_edge("func_analysis","sdk_write")
workflow.add_edge(START, "func_analysis")
tool_infograph = workflow.compile(checkpointer=checkpointer)


In [68]:
class toolcollector(MessagesState):
    total_code: List[str]
    compiled_code: str
test_message = """
def get_weather(location: str):
    \"\"\"Call to get the current weather.\"\"\"
    if location.lower() in ["sf", "san francisco"]:
        return "It's 60 degrees and foggy."
    else:
        return "It's 90 degrees and sunny."


@tool
def get_coolest_cities():
    \"\"\"Get a list of coolest cities\"\"\"
    return "nyc, sf"

tools = [get_weather, get_coolest_cities]

# Bind the model(llm) with tools
model_with_tools = ChatAnthropic(
    model="claude-3-haiku-20240307", temperature=0
).bind_tools(tools)

# Generate a tool node.
tool_node = ToolNode(tools)

# conditional edge
def should_continue(state: MessagesState):
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END
"""
tool_desc_prompt = """
You are an AI assistant designed to analyze Python code. Your task is to identify all function definitions in the provided Python snippet that are decorated with @tool. You must return a dictionary where:
- The keys are the names of the identified functions.
- You only need to pick up a function if it is decorated with '@tool' or '@tool' just preceeds the function. Otherwise leave the function alone
- The values are descriptions of what each function is supposed to do. If a function contains a docstring, extract it as the description. If a docstring is missing, infer the function's purpose from its structure and comments.
Example Input:
@tool
def calculate_area(length, width):
    "Calculates the area of a rectangle."
    return length * width

@tool
def greet(name):
    return f"Hello, {{name}}!"


Expected Output:
{{
    "calculate_area": "Calculates the area of a rectangle.",
    "greet": "Greets a user by name."
}}


Instructions:
- Identify functions that have the @tool decorator.
- Extract function names and descriptions (either from docstrings or inferred).
- Return the output as a structured JSON.
- Please only return a json object that can be converted into a json directly. DO NOT RETURN ANYTHING OTHER THAN A JSON

Python code:
<code>
{code}
</code>

"""

tool_compile_prompt = """
You are python code writing expert. You are given 2 snippets of code, your job is to combine them. 
The first snippet of code contains a compilable code with some functions compilable but empty. 
The second snippet of code contains the defination of those functions. 
Please fo through the second snippet of code, match the function in the first snippet and replace the functional definition written in the first snippet with one found in second snippet

Please only return compilable python code
Here are the code snippets:
<code_snippet1>
{complete_code}
</code_snippet1>
<code_snippet2>
{functions}
</code_snippet2>
"""

import uuid

def graph_map_step(state: toolcollector):
    # Extract nodes and edges from json_objects
    current_code = state['messages']
    print(current_code)
    response_1 = llm.invoke([SystemMessage(content=tool_desc_prompt.format(code = current_code))])
    json_objects = json.loads(response_1.content)
    print(json_objects)
    uuid_str = uuid.uuid4()
    config = {"configurable": {"thread_id": str(uuid_str)}}
    send = []
    for key in json_objects:
        print(key + " " + json_objects[key])
        for output in tool_infograph.stream({"objective":key + " " + json_objects[key], "name": key, "input":[], "output": [], "name_toolkit": "", "code":""}, config, stream_mode="updates"):
            print(output)
        send.append(tool_infograph.get_state(config).values["code"])
    return {
        "total_code": send,
        "compiled_code" : current_code
    }

def compile_code(state: toolcollector):
    tool_code_list = state['total_code']
    normal_code = state['messages']
    full_tool_code = " ".join(tool_code_list)
    response = llm.invoke([SystemMessage(content=tool_compile_prompt.format(complete_code = normal_code, functions = full_tool_code))])
    return {
        "messages": response.content
    }




In [69]:
workflow1 = StateGraph(toolcollector)
# workflow1.add_node("tool_infograph", tool_infograph)
workflow1.add_node("graph_map_step", graph_map_step)
workflow1.add_node("compile_code", compile_code)

workflow1.add_edge(START, "graph_map_step")
workflow1.add_edge("graph_map_step","compile_code")
workflow1.add_edge("compile_code", END)
tool_compile = workflow1.compile()


In [70]:
workflow = StateGraph(AgentBuilderState)
workflow.add_node("requirement_analysis", requirement_analysis_node)
workflow.add_node("evaluate_against_architecture", evaluate_against_architecture)
workflow.add_node("best_architecture", best_architecture)
workflow.add_node("agent_kernel_builder", agent_kernel_builder)
workflow.add_node("code_to_json", code_to_json_node)
workflow.add_node("json_update", json_better_node)
workflow.add_node("json_to_code", compiler_graph)
workflow.add_node("tool_compile",tool_compile)
# workflow.add_node("reflection", final_agent)
@workflow.add_node
def add_tool_message(state: AgentBuilderState):
    
   
    return {
        "messages": [
            ToolMessage(
                content="Requirements generated!",
                tool_call_id=state["messages"][-1].tool_calls[0]["id"],
            )
        ]
    }

# workflow.add_edge("reflection", END)
workflow.add_edge("tool_compile", END)
workflow.add_edge("json_to_code", "tool_compile")
workflow.add_edge("json_update", "json_to_code")
workflow.add_edge("code_to_json", "json_update")
workflow.add_edge("agent_kernel_builder", "code_to_json")
workflow.add_edge("best_architecture", "agent_kernel_builder")
workflow.add_edge("evaluate_against_architecture", "best_architecture")
workflow.add_conditional_edges("add_tool_message", architecture_evaluation_map_node,["evaluate_against_architecture"])
workflow.add_conditional_edges("requirement_analysis", route_state, ["add_tool_message", "requirement_analysis", END])
workflow.add_edge(START, "requirement_analysis")
infograph = workflow.compile()

In [71]:
import uuid

cached_human_responses = ["hi!", "rag prompt", "1 rag, 2 none, 3 no, 4 no", "red", "q"]
cached_response_index = 0
config = {"configurable": {"thread_id": str(uuid.uuid4())}}
while True:
    try:
        user = input("User (q/Q to quit): ")
    except:
        user = cached_human_responses[cached_response_index]
        cached_response_index += 1
    print(f"User (q/Q to quit): {user}")
    if user in {"q", "Q"}:
        print("AI: Byebye")
        break
    output = None
    for output in infograph.stream(
        {"messages": [HumanMessage(content=user)]}, config=config, stream_mode="updates"
    ):
        last_message = next(iter(output.values()))["messages"][-1]
        last_message.pretty_print()

    if output and "prompt" in output:
        print("Done!")
        

User (q/Q to quit): 1. Primary objective is handle customers, resolve their issues, fetch information for them and conflict to resolution 2. Use cases would be updating customer information, providing customers with the data they ask for, asking customers for their requirements and suggesting them products 3. example 1: input: what is the price of product x? output:  price of x is 300$ example 2: input: Please update my address to "delhi" output: Address updated, here are your new user details-> Name: abc, age 24, Address: Delhi Example 3: I am opening up a bakery and need to setup my supply chain and billing output: We have product x that will output your supply chain monitoring and product Y that would automate your billing and accoutning along with, I would also suggest product z which helps with customer out reach and is generally used by customers opening new business and fits your use case


2025-05-15 21:57:02,392 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


================================== Ai Message ==================================
Tool Calls:
  AgentInstructions (call_wVryVUy3Sx7RlOI79Za5edSK)
 Call ID: call_wVryVUy3Sx7RlOI79Za5edSK
  Args:
    objective: Handle customers, resolve their issues, fetch information for them and conflict to resolution
    usecases: ['Updating customer information', 'Providing customers with the data they ask for', 'Asking customers for their requirements', 'Suggesting products to customers']
    examples: Example 1: input: what is the price of product x? output: price of x is 300$; Example 2: input: Please update my address to "delhi" output: Address updated, here are your new user details-> Name: abc, age 24, Address: Delhi; Example 3: input: I am opening up a bakery and need to setup my supply chain and billing output: We have product x that will output your supply chain monitoring and product Y that would automate your billing and accounting along with, I would also suggest product z which helps with

2025-05-15 21:57:06,475 - INFO - HTTP Request: GET https://langchain-ai.github.io/langgraph/tutorials/multi_agent/multi-agent-collaboration "HTTP/1.1 301 Moved Permanently"
2025-05-15 21:57:06,482 - INFO - HTTP Request: GET https://langchain-ai.github.io/langgraph/tutorials/plan-and-execute/plan-and-execute "HTTP/1.1 301 Moved Permanently"
2025-05-15 21:57:06,488 - INFO - HTTP Request: GET https://langchain-ai.github.io/langgraph/tutorials/multi_agent/agent_supervisor "HTTP/1.1 301 Moved Permanently"
2025-05-15 21:57:06,490 - INFO - HTTP Request: GET https://langchain-ai.github.io/langgraph/tutorials/multi_agent/hierarchical_agent_teams "HTTP/1.1 301 Moved Permanently"
2025-05-15 21:57:06,497 - INFO - HTTP Request: GET https://langchain-ai.github.io/langgraph/tutorials/self-discover/self-discover "HTTP/1.1 301 Moved Permanently"
2025-05-15 21:57:06,512 - INFO - HTTP Request: GET https://langchain-ai.github.io/langgraph/tutorials/multi_agent/multi-agent-collaboration/ "HTTP/1.1 200 OK"


================================== Ai Message ==================================

Evaluated architecture https://langchain-ai.github.io/langgraph/tutorials/multi_agent/hierarchical_agent_teams, arch_name: Hierarchical Agent Teams
================================== Ai Message ==================================

Evaluated architecture https://langchain-ai.github.io/langgraph/tutorials/plan-and-execute/plan-and-execute, arch_name: Plan-and-Execute Agent Architecture
================================== Ai Message ==================================

Evaluated architecture https://langchain-ai.github.io/langgraph/tutorials/multi_agent/agent_supervisor, arch_name: Multi-agent Supervisor Architecture
================================== Ai Message ==================================

Evaluated architecture https://langchain-ai.github.io/langgraph/tutorials/self-discover/self-discover, arch_name: Self-Discover Agent
================================== Ai Message ==================================

E

...

{'sdk_write': {'name_toolkit': 'lemonai'}}


2025-05-15 21:59:40,135 - INFO - HTTP Request: GET https://python.langchain.com/docs/integrations/tools/lemonai "HTTP/1.1 308 Permanent Redirect"
2025-05-15 21:59:40,433 - INFO - HTTP Request: GET https://python.langchain.com/docs/integrations/tools/lemonai/ "HTTP/1.1 200 OK"
2025-05-15 21:59:41,195 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'code_write': {'code': '```python\nimport os\nfrom lemonai import execute_workflow\nfrom langchain_openai import OpenAI\n\ndef suggestion_node(customer_needs: str, suggestion_agent: object) -> list:\n    """\n    Suggests products to customers based on their needs and interacts with a suggestion agent to provide tailored recommendations.\n\n    Parameters:\n    customer_needs (str): The needs of the customer for product suggestions.\n    suggestion_agent (object): The suggestion agent to interact with for recommendations.\n\n    Returns:\n    list: A list of suggested products based on customer needs.\n    """\n    # Define API keys\n    os.environ["OPENAI_API_KEY"] = "*INSERT OPENAI API KEY HERE*"\n    \n    # Define the prompt for the suggestion agent\n    prompt = f"Based on the following customer needs: \'{customer_needs}\', suggest relevant products."\n    \n    # Initialize the model\n    model = OpenAI(temperature=0)\n    \n    # Execute the workflow to get product suggestions\

2025-05-15 21:59:45,281 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


================================ Human Message =================================

```python
import os
from langchain import Graph, InMemoryCheckpointer
from langchain.chat_models import ChatOpenAI
from langchain.schema import BaseMessage, HumanMessage, SystemMessage, AIMessage
from langchain.output_parsers import StructuredOutputParser
from pydantic import BaseModel
from typing import List, Dict, Any
from langchain_agentql.tools import ExtractWebDataTool
from lemonai import execute_workflow
from langchain_openai import OpenAI

# Define MessagesState as a Pydantic model for structured output
class Message(BaseModel):
    role: str
    content: str

class MessagesState(BaseModel):
    messages: List[Message]

# Initialize the LLM
llm = ChatOpenAI(model_name="gpt-4", temperature=0)

# Inquiry node implementation using structured output with langchain
def inquiry_node(state: MessagesState) -> MessagesState:
    """This node handles customer inquiries and invokes the inquiry agent."""
    #